In [5]:
#Modulo para las funciones que se utilizan en la optimización

def closest_point_and_reorder(home, points):

    # Convertir las coordenadas de los puntos a un numpy array y trabajar solo con las coordenadas (últimas 2 columnas)
    coords = np.array(points[:, -2:], dtype=float)
    
    # Convertir el punto fijo a un array numpy
    home = np.array(home, dtype=float)
    
    # Calcular las distancias euclidianas entre el punto fijo y cada punto de 'coords'
    distances = np.linalg.norm(coords - home, axis=1)
    
    # Encontrar el índice del punto con la menor distancia
    closest_index = np.argmin(distances)
    
    # Reordenar los puntos colocando el más cercano en la primera posición
    reordered_points = np.vstack([points[closest_index], np.delete(points, closest_index, axis=0)])
    
    return reordered_points


class CustomSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        # Generar n_samples rutas aleatorias
        samples = []
        for _ in range(n_samples):
            # Crear una ruta aleatoria (permutación de las paradas)
            route = np.random.permutation(n_stops)
            #print(f"n_paradas {n_stops}")
            samples.append(route)
            #print(f"samples {samples}")
        return np.array(samples)

 #Visualización de los resultados
def visualize_3(problem, x, n, fig=None, ax=None, show=True, label=True):
    with plt.style.context('ggplot'):
        # Asegúrate de que x sea un array unidimensional
        x = np.asarray(x)  # Convierte x a un array de numpy

        if x.ndim == 1:  # Asegúrate de que es un vector
            x = x.flatten()  # Aplana el array para asegurarte de que sea 1D

        if fig is None or ax is None:
            fig, ax = plt.subplots()

        # Plot de las ciudades usando scatter plot
        ax.scatter(problem.stops[:, 0], problem.stops[:, 1], s=150)
        if label:
            # Anota las ciudades
            for i, c in enumerate(problem.stops):
                ax.annotate(str(i), xy=c, fontsize=10, ha="center", va="center", color="white")

        # Plotea la línea en el camino
        for i in range(len(x) - 1):
            current = int(x[i])  # Asegúrate de convertir a entero
            next_ = int(x[i + 1])  # Asegúrate de convertir a entero
            ax.plot(problem.stops[[current, next_], 0], problem.stops[[current, next_], 1], 'r--')

        fig.suptitle(f"Route length: {problem.get_route_length(x)} km \nRoute time: {problem.get_route_length(x) * 60 / velocidad_media} min")

        if show:
            plt.show()

#Funciones para el cálculo de curvas de Bezier

from scipy.special import comb

def get_bezier_parameters(X, Y, smooth_factor, degree=3):
    """ Least square qbezier fit using penrose pseudoinverse.

    Parameters:

    X: array of x data.
    Y: array of y data. Y[0] is the y point for X[0].
    degree: degree of the Bézier curve. 2 for quadratic, 3 for cubic.

    Based on https://stackoverflow.com/questions/12643079/b%C3%A9zier-curve-fitting-with-scipy
    and probably on the 1998 thesis by Tim Andrew Pastva, "Bézier Curve Fitting".
    """
    if degree < 1:
        raise ValueError('degree must be 1 or greater.')

    if len(X) != len(Y):
        raise ValueError('X and Y must be of the same length.')

    #if len(X) < degree + 1:
    #    raise ValueError(f'There must be at least {degree + 1} points to '
    #                     f'determine the parameters of a degree {degree} curve. '
    #                     f'Got only {len(X)} points.')

    def bpoly(n, t, k):
        """ Bernstein polynomial when a = 0 and b = 1. """
        return t ** k * (1 - t) ** (n - k) * comb(n, k)
        #return comb(n, i) * ( t**(n-i) ) * (1 - t)**i

    def bmatrix(T):
        """ Bernstein matrix for Bézier curves. """
        return np.matrix([[bpoly(degree, t, k) for k in range(degree + 1)] for t in T])

    def least_square_fit(points, M, smooth_factor):
        M_ = np.linalg.pinv(M)
        
        smooth_matrix = np.eye(M.shape[1])
        for i in range(1, M.shape[1]):
            smooth_matrix[i, i - 1] = -1  # Penaliza diferencias grandes entre puntos de control adyacentes

        # Combine the original fitting problem with the smoothness constraint
        augmented_matrix = np.vstack([M, smooth_factor * smooth_matrix])
        augmented_points = np.vstack([points, np.zeros((smooth_matrix.shape[0], points.shape[1]))])
        
        return np.linalg.pinv(augmented_matrix) @ augmented_points

    T = np.linspace(0, 1, len(X))
    M = bmatrix(T)
    points = np.array(list(zip(X, Y)))
    
    final = least_square_fit(points, M, smooth_factor).tolist()
    final[0] = [X[0], Y[0]]
    final[len(final)-1] = [X[len(X)-1], Y[len(Y)-1]]
    return final

def bernstein_poly(i, n, t):
    """
     The Bernstein polynomial of n, i as a function of t
    """
    return comb(n, i) * ( t**(n-i) ) * (1 - t)**i


def bezier_curve(points, nTimes=50):
    """
       Given a set of control points, return the
       bezier curve defined by the control points.

       points should be a list of lists, or list of tuples
       such as [ [1,1], 
                 [2,3], 
                 [4,5], ..[Xn, Yn] ]
        nTimes is the number of time steps, defaults to 1000

        See http://processingjs.nihongoresources.com/bezierinfo/
    """

    nPoints = len(points)
    xPoints = np.array([p[0] for p in points])
    yPoints = np.array([p[1] for p in points])

    t = np.linspace(0.0, 1.0, nTimes)

    polynomial_array = np.array([ bernstein_poly(i, nPoints-1, t) for i in range(0, nPoints)   ])

    xvals = np.dot(xPoints, polynomial_array)
    yvals = np.dot(yPoints, polynomial_array)

    return xvals, yvals

# Función para formatear el XML
def prettify(element):
    """ Devuelve una versión 'bonita' del XML """
    rough_string = ET.tostring(element, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

NameError: name 'Sampling' is not defined

In [ ]:
        with open(f'/home/dacya-iagesbloom/Documents/paparazzi/var/aircrafts/Rover_Avelino/flight_plan.xml', 'w', encoding='utf-8') as f:
            f.write('<!DOCTYPE flight_plan SYSTEM "../flight_plan.dtd">\n')
            f.write(prettify(root))
        print("FLIGHT PLAN GUARDADO")
        print("Waypoints reescritos y guardados")

In [6]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

Archivo = 'flight_plan_pruebaQT'
Controlador = 'rover2f405WSE'

#Ahora vamos a modificar el archivo conf.xml que se encuentra en la ruta paparazzi/conf/airframes/UCM para que cuando se abra paparazzi de nuevo el flight plan y el airframe sean los que pongamos aqui

# Ruta del archivo XML que deseas modificar
tree = ET.parse('/home/dacya-iagesbloom/Documents/paparazzi/conf/airframes/UCM/conf.xml')
root = tree.getroot()

new_flight_plan = f'{Archivo}_opt'
new_airframe = f'{Controlador}_opt'
aircraft_name = 'Rover_Avelino'
# Parsear el archivo XML

#Buscar el nodo correspondiente al aircraft
for aircraft in root.iter('aircraft'):
    # Si el nombre del aircraft coincide, actualizar los valores
    if aircraft.get('name') == aircraft_name:
        aircraft.set('airframe', new_airframe)
        aircraft.set('flight_plan', new_flight_plan)
# Guardar los cambios en el archivo XML
output_path_comp = '/home/dacya-iagesbloom/Documents/paparazzi/conf/airframes/UCM/conf.xml'
tree.write(output_path_comp, encoding='utf-8', xml_declaration=True)

In [2]:
import os
import subprocess
import time
import psutil

PAPARAZZI_DIR = "/home/dacya-iagesbloom/Documents/paparazzi"

# Cambiar al directorio de Paparazzi
os.chdir(PAPARAZZI_DIR)

#Hacer un make del nuevo paparazzi

# Definir el directorio del proyecto y el comando make
project_dir = '/home/dacya-iagesbloom/Documents/paparazzi'  # Cambia esto a la ruta de tu proyecto
make_command = ['make']  # Puedes añadir otros argumentos si los necesitas

# Cambiar al directorio del proyecto
try:
    subprocess.run(make_command, cwd=project_dir, check=True)
    print("Make ejecutado exitosamente")
except subprocess.CalledProcessError as e:
    print(f"Error al ejecutar make: {e}")

def close_paparazzi_and_gcs():
    # Cerrar procesos relacionados con el puerto 2010
    try:
        # Buscar procesos que están usando el puerto 2010
        result = subprocess.run(['sudo', 'lsof', '-t', '-i', ':2010'], stdout=subprocess.PIPE)
        pid_list = result.stdout.decode().splitlines()

        if pid_list:
            for pid in pid_list:
                # Matar cada proceso encontrado en el puerto 2010
                subprocess.run(['sudo', 'kill', '-9', pid])
                print(f"Proceso con PID {pid} terminado.")
        else:
            print("No se encontraron procesos usando el puerto 2010.")

        # Cerrar Paparazzi GCS y otros procesos gráficos
        for proc in psutil.process_iter(['pid', 'name']):
            if 'paparazzi' in proc.info['name'].lower():
                proc.kill()  # Matar el proceso de Paparazzi
                print(f"Proceso Paparazzi con PID {proc.info['pid']} terminado.")
            
            # Cerrar el GCS si está en ejecución (por si acaso)
            if 'groundstation' in proc.info['name'].lower() or 'gcs' in proc.info['name'].lower():
                proc.kill()  # Matar el proceso de la estación de tierra
                print(f"Proceso Estación de Tierra (GCS) con PID {proc.info['pid']} terminado.")

    except Exception as e:
        print(f"Error al cerrar procesos: {e}")

# Llamar a la función para cerrar los procesos antes de iniciar uno nuevo
close_paparazzi_and_gcs()

# Especificar la ruta completa del ejecutable 'paparazzi'
PAPARAZZI_EXECUTABLE = os.path.join(PAPARAZZI_DIR, "paparazzi")

# Reiniciar la GCS después de la compilación exitosa
subprocess.run([PAPARAZZI_EXECUTABLE, "&"])


------------------------------------------------------------
Building Paparazzi version v6.2_unstable-249-g3563cb5c9-dirty
------------------------------------------------------------
[ -L conf/control_panel.xml ] || [ -f conf/control_panel.xml ] || cp conf/control_panel_example.xml conf/control_panel.xml
make -C sw/ext pprzlink.update
make[1]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink'
make[2]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/ocaml'
Check for already installed pprzlink ocaml lib
 -> found lib version 2.0 while installing 2.0
Build bytecode lib
Build native lib
INSTALL at location /home/dacya-iagesbloom/Documents/paparazzi/var/lib/ocaml
make[2]: Leaving directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/ocaml'
make[2]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/python'
# Install correct version python library to DESTDIR
IN

make[2]: *** [Makefile:80: lib] Error 1
make[1]: *** [Makefile:58: libopencm3.build] Error 2
make: *** [Makefile:176: ext] Error 2
sudo: a terminal is required to read the password; either use the -S option to read from standard input or configure an askpass helper


conf restored.
conf saved to /home/dacya-iagesbloom/Documents/paparazzi/conf/airframes/UCM/conf.xml


CompletedProcess(args=['/home/dacya-iagesbloom/Documents/paparazzi/paparazzi', '&'], returncode=0)

In [4]:
import subprocess

# Definir el directorio del proyecto y el comando make
project_dir = '/home/dacya-iagesbloom/Documents/paparazzi'  # Cambia esto a la ruta de tu proyecto
make_command = ['make']  # Puedes añadir otros argumentos si los necesitas

# Cambiar al directorio del proyecto
try:
    subprocess.run(make_command, cwd=project_dir, check=True)
    print("Make ejecutado exitosamente")
except subprocess.CalledProcessError as e:
    print(f"Error al ejecutar make: {e}")


------------------------------------------------------------
Building Paparazzi version v6.2_unstable-249-g3563cb5c9-dirty
------------------------------------------------------------
[ -L conf/control_panel.xml ] || [ -f conf/control_panel.xml ] || cp conf/control_panel_example.xml conf/control_panel.xml
make -C sw/ext pprzlink.update
make[1]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink'
make[2]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/ocaml'
Check for already installed pprzlink ocaml lib
 -> found lib version 2.0 while installing 2.0
Build bytecode lib
Build native lib
INSTALL at location /home/dacya-iagesbloom/Documents/paparazzi/var/lib/ocaml
make[2]: Leaving directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/ocaml'
make[2]: Entering directory '/home/dacya-iagesbloom/Documents/paparazzi/sw/ext/pprzlink/lib/v2.0/python'
# Install correct version python library to DESTDIR
IN

In [4]:
#Open flight plan editor

from PyQt5.QtCore import QProcess

# Ruta del ejecutable del Flight Plan Editor
flight_plan_editor_path = '/home/dacya-iagesbloom/Documents/paparazzi/conf/tools/flight_plan_editor.xml'

# Crear un proceso para ejecutar el Flight Plan Editor
process = QProcess()
process.start(flight_plan_editor_path)

# Verificar si el proceso se ha iniciado correctamente
#if process.waitForStarted():
#    print("Flight Plan Editor abierto exitosamente.")
#else:
    #print("No se pudo abrir el Flight Plan Editor.")